<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Comparasión de Clasificadores</strong></h1>
  <h3><strong>Walter y Heri</strong></h3>
</div>

**Objetivo:** El objetivo de esta actividad es que cada equipo cree una función que permita manejar diferentes formas de representación de clasificaciones.

**Instrucciones:**
- Construye una clase en Python que reciba una clasificación en representación vectorial, y construya la representación matricial y de conjunto de conjuntos de la clasificación. Haz un método para cada conversión:  
    - vectorial -> matricial.  
    - vectorial -> conjunto de conjuntos.  
- Guarda las tres representaciones como variables ocultas usando la nomenclatura "__parameter".  
- Crea tres métodos getter para devolver cada representación.  
- Construye un método para calcular los siguientes valores:  
    - Entropía de una clasificación.  
    - La entropía condicional de una clasificación dada otra clasificación.  
    - La información mutua de una clasificación dada otra clasificación.  
    - La variación de información de una clasificación dada otra clasificación.  

In [155]:
import numpy as np

class MyClass():
    def __init__(self, vector):
        self.__vector = vector
        self.__matrix = self.create_matrix()
        self.__sets = self.create_sets()

    def create_matrix(self):
        # Get labels
        labels = [] 

        for label in self.__vector:
            if label not in labels:
                labels.append(label) 

        # Create an empty matrix
        matrix = [ [0] * len(labels) for i in range(len(self.__vector)) ]

        # Fill the matrix
        for label in range(len(self.__vector)): # for each label in the vector
            matrix[label][labels.index(self.__vector[label])] = 1 # set the value to 1
            # labels.index(self.__vector[label]) returns the index of the label in the labels list

        return matrix
    
    def create_sets(self):
        return None
    
    # ---
    # Crea tres métodos getter para devolver cada representación. 

    def get_vector(self):
        return self.__vector
    
    def get_matrix(self):
        return self.__matrix
    
    def get_sets(self):
        return self.__sets

    
    # ---
    # Construye un método para calcular los siguientes valores:
    # - Entropía de una clasificación.


    def entropy(self):
        entropy = 0

        p = [] # create a list of probabilities
        for label in self.__matrix[1]: # for each label in the matrix
            p.append(self.__vector.count(label) / len(self.__vector)) # count the number of times the label appears in the vector
        
        for prob in p:
            term = -prob * np.log2(prob)
            entropy += term
            
        return entropy
    
    # - La entropía condicional de una clasificación dada otra clasificación.

    def conditional_entropy(self, other):
        conditional_entropy = 0

        p = [] # create a list of probabilities

        # Intersection between the two classifications
        intersection = []

        for i in range(len(self.__vector)):
            if self.__vector[i] == other[i]:
                intersection.append(self.__vector[i])
        
        for label in self.__matrix[1]: # for each label in the intersection
            p.append(intersection.count(label) / len(intersection))

        for prob in p:
            term = -prob * np.log2(prob)
            conditional_entropy += term

        print("Intersection:", intersection, "Probabilities:", p)
        return conditional_entropy
        # example usage: my_class.conditional_entropy([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

    
    # - La información mutua de una clasificación dada otra clasificación.

    def mutual_information(self, other):
        return self.entropy() - self.conditional_entropy(other)
    


In [156]:
vector = (0, 0, 0, 1, 1, 1, 2, 2, 2)
my_class = MyClass(vector)

my_class.create_matrix()


[[1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1]]

In [157]:
my_class.entropy()

1.584962500721156

In [158]:
my_class.conditional_entropy((0, 0, 0, 1, 1, 1, 2, 2, 2)) 

Intersection: [0, 0, 0, 1, 1, 1, 2, 2, 2] Probabilities: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


1.584962500721156

In [159]:
my_class.mutual_information((0, 0, 0, 1, 1, 1, 2, 2, 2))

Intersection: [0, 0, 0, 1, 1, 1, 2, 2, 2] Probabilities: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


0.0